In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd
from plot_helpers import plot_ies_results

In [ ]:
t_d = "template"
if not os.path.exists(t_d):
    raise Exception("need to run simple_pstfrom notebook")

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"at.pst"))
pst.npar,pst.nobs

In [ ]:
pst.pestpp_options

In [ ]:
if "ies_par_en" not in pst.pestpp_options:
    raise Exception("prior par ensemble not found")

In [ ]:
pst.pestpp_options = {"ies_par_en":pst.pestpp_options["ies_par_en"]}
pst.pestpp_options["ies_num_reals"] = 50
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options

In [ ]:
pst.control_data.noptmax = 5

In [ ]:
pst.obs_groups

In [ ]:
weighted_groups = pst.nnz_obs_groups
weighted_groups.sort()
weighted_groups

In [ ]:
wts = [1e-30,.8,.2]

In [ ]:
with open(f'{t_d}/phi_factors.csv', 'w') as ofp:
    [ofp.write(f'{g},{w}\n') for g,w in zip(weighted_groups,wts)]

In [ ]:
pst.pestpp_options['ies_phi_factor_file']='phi_factors.csv'

In [ ]:
pst.write(os.path.join(t_d,"at.pst"),version=2)

In [ ]:
m_d = "master_ies_simple"

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')

In [ ]:
plot_ies_results(m_d)

In [ ]:
pst.pestpp_options["ies_no_noise"] = True
pst.write(os.path.join(t_d,"at.pst"),version=2)
m_d = "master_ies_nonoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')
plot_ies_results(m_d,tag="no noise")

In [ ]:
obs = pst.observation_data
hobs = obs.loc[(obs.weight>0) & (obs.obgnme=="headgroup"),:]
hobs.obsnme.tolist()
assert len(hobs) > 0
std = hobs.standard_deviation.max()
std

In [ ]:
offsets = np.random.normal(0.0,std,pst.ies.noise.shape[0])
plt.hist(offsets)
offsets.shape

In [ ]:
ovals = obs.loc[pst.nnz_obs_names,"obsval"].copy()
noise = pst.ies.noise.copy()
for i,idx in enumerate(noise.index):
    if idx == "base":
        offsets[i] = 0.0
    noise.loc[idx,hobs.obsnme] = ovals[hobs.obsnme] + offsets[i]
noise.to_csv(os.path.join(t_d,"corrnoise.csv"))
#print(noise.loc[:,hobs.obsnme[0]]-ovals.loc[hobs.obsnme[0]])
#print(noise.loc[:,hobs.obsnme[1]]-ovals.loc[hobs.obsnme[1]])

In [ ]:
pst.pestpp_options["ies_obs_en"] = "corrnoise.csv"

In [ ]:
pst.pestpp_options.pop("ies_no_noise")

In [ ]:
pst.write(os.path.join(t_d,"at.pst"),version=2)
m_d = "master_ies_corrnoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')
plot_ies_results(m_d,tag="corr noise")